In [10]:
# Gerekli kütüphaneleri yükleyelim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

X_train = "C:/Users/ASUS/Desktop/Regression of Used Car Prices/train.csv"

y_train = "C:/Users/ASUS/Desktop/Regression of Used Car Prices/test.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [12]:
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# ---------- LightGBM Modeli ----------

# LightGBM modelini tanımla
lgb_model = lgb.LGBMRegressor(
    subsample=0.9,
    num_leaves=31,
    n_estimators=750,
    min_child_samples=30,
    max_depth=10,
    learning_rate=0.01,
    colsample_bytree=0.8
)

# LightGBM modelini eğitim verisi üzerinde eğit
lgb_model.fit(X_train, y_train)

# Validation seti üzerindeki LightGBM tahminleri
y_pred_lgb = lgb_model.predict(X_valid)

# Performans değerlendirmesi (RMSE) - LightGBM
rmse_lgb = mean_squared_error(y_valid, y_pred_lgb, squared=False)
print(f"Validation RMSE (LightGBM): {rmse_lgb}")

# ---------- CatBoost Modeli ----------

# CatBoost modelini tanımla
catboost_model = CatBoostRegressor(
    iterations=750,
    depth=10,
    learning_rate=0.01,
    l2_leaf_reg=3,
    loss_function='RMSE',
    custom_metric=['RMSE'],
    verbose=0  # Hata ayıklamayı kapat
)

# CatBoost modelini eğitim verisi üzerinde eğit
catboost_model.fit(X_train, y_train)

# Validation seti üzerindeki CatBoost tahminleri
y_pred_catboost = catboost_model.predict(X_valid)

# Performans değerlendirmesi (RMSE) - CatBoost
rmse_catboost = mean_squared_error(y_valid, y_pred_catboost, squared=False)
print(f"Validation RMSE (CatBoost): {rmse_catboost}")

# ---------- Ensemble (Birleştirme) ----------

# LightGBM ve CatBoost tahminlerinin ortalamasını alarak ensemble yapma
y_pred_ensemble = (y_pred_lgb + y_pred_catboost) / 2

# Ensemble modelinin performans değerlendirmesi (RMSE)
rmse_ensemble = mean_squared_error(y_valid, y_pred_ensemble, squared=False)
print(f"Validation RMSE (Ensemble): {rmse_ensemble}")


ValueError: Expected 2D array, got scalar array instead:
array=C:/Users/ASUS/Desktop/Regression of Used Car Prices/train.csv.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [4]:
# 6. Batch'ler Halinde Test Setinde Tahminler
batch_size = 1000
all_predictions = pd.DataFrame()

for i in range(0, len(test_data), batch_size):
    batch = test_data[i:i+batch_size]
    batch_predictions = best_model.predict(batch.drop(columns=['id']))
    
    batch_output = pd.DataFrame({'id': batch['id'], 'price': batch_predictions})
    all_predictions = pd.concat([all_predictions, batch_output])

# 7. Sonuçları CSV Dosyasına Kaydetme
all_predictions.to_csv('predictions.csv', index=False)